# 7장. 멀티 에이전트 패턴

In [ ]:
!pip install langchain langchain-community langchain-openai langgraph

In [ ]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')

---
## 코드 7-1 Reflection 패턴: 자기 성찰 에이전트

# Reflection 패턴: 자기 성찰 에이전트

이 노트북에서는 **Reflection 패턴**을 사용하여 **스스로 결과물을 개선**하는 에이전트를 만듭니다.

## Reflection이란?

```
┌────────────────────────────────────────────────────────────────────┐
│                    Reflection (자기 성찰) 패턴                     │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반적인 생성:                                                   │
│   질문 → LLM → 답변 (1회 생성 후 끝)                              │
│                                                                    │
│   Reflection 패턴:                                                 │
│   질문 → LLM(생성) → 결과물                                       │
│            ↑           │                                          │
│            │           ▼                                          │
│            └─── LLM(평가) ← 개선할 점 찾기                        │
│                                                                    │
│   → 생성과 평가를 반복하여 품질 향상!                             │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 아키텍처

```
┌────────────────────────────────────────────────────────────────────┐
│                    Reflection 그래프 구조                          │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   START                                                            │
│     │                                                              │
│     ▼                                                              │
│   ┌──────────┐                                                     │
│   │ generate │  ← 에세이/코드/답변 생성                           │
│   └────┬─────┘                                                     │
│        │                                                           │
│     should_continue?                                               │
│        │╲                                                          │
│        │ ╲ (계속)                                                  │
│        │  ╲                                                        │
│   (종료) ▼  ▼                                                      │
│   END    ┌──────────┐                                              │
│          │ reflect  │  ← 결과물 평가 및 피드백                    │
│          └────┬─────┘                                              │
│               │                                                    │
│               └──────────────────────────────────────┐             │
│                                                      │             │
│                                                  generate          │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. LLM 설정

In [ ]:
from langchain_ollama import ChatOllama

model = ChatOllama(model='llama3.2')

print("✅ LLM 설정 완료")

# 3. State 정의

In [ ]:
from typing import Annotated, TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

class State(TypedDict):
    """
    Reflection 패턴의 상태
    
    messages: 대화 기록
      - 사용자 요청
      - 생성된 결과물 (AI)
      - 평가 피드백 (Human으로 변환)
      - 개선된 결과물 (AI)
      - ... 반복
    """
    messages: Annotated[list[BaseMessage], add_messages]

print("✅ State 정의 완료")

# 4. 프롬프트 정의

생성과 평가에 사용할 시스템 프롬프트를 정의합니다.

In [ ]:
from langchain_core.messages import SystemMessage

# 생성용 프롬프트
generate_prompt = SystemMessage(
    content='당신은 훌륭한 3단락 에세이를 작성하는 임무를 가진 에세이 어시스턴트입니다.'
    '사용자의 요청에 맞춰 최상의 에세이를 작성하세요.'
    '사용자가 비평을 제공하면, 이전 시도에 대한 수정 버전을 응답하세요.'
)

# 평가용 프롬프트
reflection_prompt = SystemMessage(
    content='당신은 에세이 제출물을 평가하는 교사입니다. 사용자의 제출물에 대해 비평과 추천을 생성하세요.'
    '길이, 깊이, 스타일 등과 같은 구체적인 요구사항을 포함한 자세한 추천을 제공하세요.'
)

print("✅ 프롬프트 정의 완료")
print("   - generate_prompt: 에세이 작성")
print("   - reflection_prompt: 에세이 평가")

# 5. 노드 정의

```
┌────────────────────────────────────────────────────────────────────┐
│                    메시지 역할 변환 트릭                            │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   reflect 노드에서 메시지 역할을 뒤집는 이유:                      │
│                                                                    │
│   [원래]                    [변환 후]                              │
│   Human: 에세이 써줘        Human: 에세이 써줘                     │
│   AI: 에세이 내용...   →    Human: 에세이 내용...                  │
│                                                                    │
│   → AI가 자신이 쓴 에세이를 '사용자가 제출한 것'처럼               │
│     평가하도록 만듦!                                               │
│                                                                    │
│   그 후 평가 결과를 HumanMessage로 반환                            │
│   → generate 노드가 피드백으로 받아들임                            │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

def generate(state: State) -> State:
    """
    생성 노드: 에세이 작성 (또는 개선)
    
    1. 시스템 프롬프트 + 대화 기록 조합
    2. LLM으로 에세이 생성
    3. 결과를 AIMessage로 반환
    """
    answer = model.invoke([generate_prompt] + state['messages'])
    return {'messages': [answer]}


def reflect(state: State) -> State:
    """
    평가 노드: 에세이 비평
    
    1. 메시지 역할 변환 (AI → Human, Human → AI)
    2. 평가 프롬프트로 비평 생성
    3. 비평을 HumanMessage로 반환 (피드백으로 사용)
    """
    # 역할 변환 매핑
    cls_map = {AIMessage: HumanMessage, HumanMessage: AIMessage}
    
    # 첫 번째 메시지(원래 요청)는 유지, 나머지는 역할 변환
    translated = [reflection_prompt, state['messages'][0]] + [
        cls_map[msg.__class__](content=msg.content) 
        for msg in state['messages'][1:]
    ]
    
    # 평가 생성
    answer = model.invoke(translated)
    
    # 평가 결과를 HumanMessage로 반환 (피드백으로 취급)
    return {'messages': [HumanMessage(content=answer.content)]}


print("✅ 노드 정의 완료")
print("   - generate: 에세이 생성/개선")
print("   - reflect: 에세이 평가")

# 6. 조건 함수 정의

In [ ]:
from langgraph.graph import END

def should_continue(state: State):
    """
    반복 여부 결정
    
    메시지 수로 반복 횟수 제어:
    - 초기: 1개 (사용자 요청)
    - 1회차 후: 3개 (요청 + 에세이 + 피드백)
    - 2회차 후: 5개 (+ 개선 에세이 + 피드백)
    - 3회차 후: 7개 → 6개 초과 → 종료
    """
    if len(state['messages']) > 6:
        return END  # 3회 반복 후 종료
    else:
        return 'reflect'  # 계속 평가

print("✅ should_continue 함수 정의 완료")
print("   3회 반복 후 종료")

# 7. 그래프 구성

In [ ]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)

# 노드 추가
builder.add_node('generate', generate)
builder.add_node('reflect', reflect)

# 엣지 추가
builder.add_edge(START, 'generate')              # 시작 → 생성
builder.add_conditional_edges('generate', should_continue)  # 조건부 분기
builder.add_edge('reflect', 'generate')          # 평가 → 생성 (루프)

graph = builder.compile()

print("✅ 그래프 컴파일 완료")

In [ ]:
# 그래프 구조 시각화
print("=== Reflection 그래프 구조 ===")
print(graph.get_graph().draw_mermaid())

# 8. Reflection 에이전트 실행

In [ ]:
# 에세이 주제 설정
initial_state = {
    'messages': [
        HumanMessage(
            content="오늘날 '어린 왕자'가 왜 중요한지에 대해 에세이를 작성하세요."
        )
    ]
}

print("=== Reflection 에이전트 실행 ===")
print(f"주제: {initial_state['messages'][0].content}\n")

iteration = 0
for output in graph.stream(initial_state):
    if 'generate' in output:
        iteration += 1
        print(f"\n{'='*60}")
        print(f"📝 생성 #{iteration}")
        print("="*60)
        content = output['generate']['messages'][-1].content
        print(f"{content[:500]}..." if len(content) > 500 else content)
        
    elif 'reflect' in output:
        print(f"\n{'─'*60}")
        print(f"🔍 평가 #{iteration}")
        print("─"*60)
        content = output['reflect']['messages'][-1].content
        print(f"{content[:500]}..." if len(content) > 500 else content)

In [ ]:
# 최종 결과 확인
result = graph.invoke(initial_state)

print("=== 최종 에세이 ===")
# 마지막 AI 메시지 찾기
for msg in reversed(result['messages']):
    if isinstance(msg, AIMessage):
        print(msg.content)
        break

---

## 정리: Reflection 패턴

### 핵심 개념

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Reflection 패턴 동작 원리                        │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   1. generate: 결과물 생성                                          │
│   2. reflect: 결과물 평가 (역할 변환 트릭 사용)                     │
│   3. generate: 피드백 반영하여 개선                                 │
│   4. 반복...                                                        │
│                                                                     │
│   핵심 트릭:                                                        │
│   • AI 응답을 Human 메시지로 변환 → '제출물'처럼 평가              │
│   • 평가 결과를 Human 메시지로 반환 → '피드백'으로 전달            │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 핵심 코드

```python
# 역할 변환
cls_map = {AIMessage: HumanMessage, HumanMessage: AIMessage}
translated = [prompt, original_request] + [
    cls_map[msg.__class__](content=msg.content) 
    for msg in messages[1:]
]

# 피드백으로 반환
return {'messages': [HumanMessage(content=answer.content)]}
```

### 활용 사례

| 사례 | 설명 |
|------|------|
| **글쓰기** | 에세이, 보고서 품질 향상 |
| **코드** | 코드 리뷰 및 개선 |
| **번역** | 번역 품질 검토 및 수정 |
| **요약** | 요약 정확도 향상 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(model='gpt-4o-mini')

# 변경
model = ChatOllama(model='llama3.2')
```

## 다음 단계

**Subgraph**를 사용하여 그래프 내에 그래프를 넣는 방법을 배웁니다. (02번 노트북)

---
## 코드 7-2 Subgraph (직접 연결): 그래프 안의 그래프

# Subgraph (직접 연결): 그래프 안의 그래프

이 노트북에서는 **Subgraph**를 사용하여 **그래프 안에 그래프를 넣는** 방법을 배웁니다.

## Subgraph란?

```
┌────────────────────────────────────────────────────────────────────┐
│                    Subgraph의 개념                                  │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   서브그래프 = 부모 그래프 안에 포함된 자식 그래프                 │
│                                                                    │
│   ┌─────────────── 부모 그래프 ───────────────┐                   │
│   │                                          │                    │
│   │   START → ┌─────────────────┐ → END      │                    │
│   │           │   서브그래프     │            │                    │
│   │           │ ┌───┐   ┌───┐  │            │                    │
│   │           │ │ A │ → │ B │  │            │                    │
│   │           │ └───┘   └───┘  │            │                    │
│   │           └─────────────────┘            │                    │
│   │                                          │                    │
│   └──────────────────────────────────────────┘                    │
│                                                                    │
│   장점:                                                            │
│   • 복잡한 로직을 모듈화                                           │
│   • 재사용 가능한 그래프 컴포넌트                                  │
│   • 코드 관리 용이                                                 │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 이번 노트북: 직접 연결 방식

```
┌────────────────────────────────────────────────────────────────────┐
│                    직접 연결 vs 함수 래핑                           │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   [직접 연결] (이번 노트북)                                        │
│   • 부모와 자식이 상태 키를 공유                                   │
│   • builder.add_node('name', subgraph)                            │
│   • 간단하지만 유연성 낮음                                         │
│                                                                    │
│   [함수 래핑] (다음 노트북)                                        │
│   • 상태 키가 달라도 됨                                            │
│   • 함수 안에서 상태 변환                                          │
│   • 더 유연함                                                      │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

# 2. State 정의

부모 그래프와 서브그래프의 상태를 정의합니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    상태 공유 구조                                   │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   부모 State:                     서브그래프 State:                │
│   ┌─────────────┐                ┌─────────────┐                  │
│   │ foo: str    │ ◄──── 공유 ────► foo: str    │                  │
│   └─────────────┘                │ bar: str    │ ← 추가 필드       │
│                                  └─────────────┘                  │
│                                                                    │
│   'foo' 키를 통해 부모 ↔ 서브그래프 통신                          │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from typing import TypedDict

# 부모 그래프의 상태
class State(TypedDict):
    foo: str  # 서브그래프와 공유되는 키

# 서브그래프의 상태
class SubgraphState(TypedDict):
    foo: str  # 부모 그래프와 공유되는 키
    bar: str  # 서브그래프 내부에서만 사용

print("✅ State 정의 완료")
print("   - State (부모): foo")
print("   - SubgraphState: foo (공유), bar (내부용)")

# 3. 서브그래프 정의

In [ ]:
from langgraph.graph import StateGraph, START

def subgraph_node(state: SubgraphState):
    """
    서브그래프의 노드
    
    공유 키인 'foo'를 수정하여 부모 그래프와 통신
    """
    print(f"  [서브그래프] 입력 foo: {state['foo']}")
    
    # foo 값에 'bar' 추가
    new_foo = state['foo'] + 'bar'
    
    print(f"  [서브그래프] 출력 foo: {new_foo}")
    return {'foo': new_foo}


# 서브그래프 빌더
subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node('subgraph_node', subgraph_node)
subgraph_builder.add_edge(START, 'subgraph_node')

# 서브그래프 컴파일
subgraph = subgraph_builder.compile()

print("✅ 서브그래프 정의 완료")

In [ ]:
# 서브그래프 단독 테스트
print("=== 서브그래프 단독 테스트 ===")
result = subgraph.invoke({'foo': 'hello', 'bar': ''})
print(f"결과: {result}")

# 4. 부모 그래프 정의 (서브그래프 직접 연결)

In [ ]:
# 부모 그래프 빌더
builder = StateGraph(State)

# 서브그래프를 노드로 직접 추가!
builder.add_node('subgraph', subgraph)  # 컴파일된 서브그래프를 노드로 추가
builder.add_edge(START, 'subgraph')

# 부모 그래프 컴파일
graph = builder.compile()

print("✅ 부모 그래프 정의 완료")
print("   서브그래프가 'subgraph' 노드로 추가됨")

In [ ]:
# 그래프 구조 시각화
print("=== 부모 그래프 구조 ===")
print(graph.get_graph().draw_mermaid())

# 5. 그래프 실행

In [ ]:
# 부모 그래프 실행
initial_state = {'foo': 'hello'}

print("=== 부모 그래프 실행 ===")
print(f"입력: {initial_state}")
print()

result = graph.invoke(initial_state)

print()
print(f"출력: {result}")
print()
print("→ 'hello' + 'bar' = 'hellobar'")

# 6. 여러 서브그래프 연결

In [ ]:
# 두 번째 서브그래프 정의
def subgraph_node2(state: SubgraphState):
    print(f"  [서브그래프2] 입력 foo: {state['foo']}")
    new_foo = state['foo'] + '!'
    print(f"  [서브그래프2] 출력 foo: {new_foo}")
    return {'foo': new_foo}

subgraph_builder2 = StateGraph(SubgraphState)
subgraph_builder2.add_node('subgraph_node2', subgraph_node2)
subgraph_builder2.add_edge(START, 'subgraph_node2')
subgraph2 = subgraph_builder2.compile()

print("✅ 두 번째 서브그래프 정의 완료")

In [ ]:
from langgraph.graph import END

# 여러 서브그래프를 연결하는 부모 그래프
multi_builder = StateGraph(State)
multi_builder.add_node('sub1', subgraph)   # 첫 번째 서브그래프
multi_builder.add_node('sub2', subgraph2)  # 두 번째 서브그래프
multi_builder.add_edge(START, 'sub1')
multi_builder.add_edge('sub1', 'sub2')
multi_builder.add_edge('sub2', END)

multi_graph = multi_builder.compile()

print("✅ 다중 서브그래프 부모 그래프 완료")

In [ ]:
# 그래프 구조 시각화
print("=== 다중 서브그래프 구조 ===")
print(multi_graph.get_graph().draw_mermaid())

In [ ]:
# 실행
print("=== 다중 서브그래프 실행 ===")
print(f"입력: {{'foo': 'hello'}}")
print()

result = multi_graph.invoke({'foo': 'hello'})

print()
print(f"출력: {result}")
print()
print("→ 'hello' + 'bar' + '!' = 'hellobar!'")

---

## 정리: Subgraph 직접 연결

### 핵심 개념

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Subgraph 직접 연결 방식                          │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   1. 서브그래프 정의 및 컴파일                                      │
│      subgraph = subgraph_builder.compile()                          │
│                                                                     │
│   2. 부모 그래프에서 노드로 추가                                    │
│      builder.add_node('name', subgraph)                             │
│                                                                     │
│   3. 공유 키를 통해 데이터 전달                                     │
│      부모 State.foo ↔ 서브그래프 State.foo                          │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 핵심 코드

```python
# 1. 공유 키가 있는 State 정의
class State(TypedDict):
    foo: str  # 공유 키

class SubgraphState(TypedDict):
    foo: str  # 공유 키
    bar: str  # 내부 키

# 2. 서브그래프 정의 및 컴파일
subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node('node', node_fn)
subgraph_builder.add_edge(START, 'node')
subgraph = subgraph_builder.compile()

# 3. 부모 그래프에서 서브그래프를 노드로 추가
builder = StateGraph(State)
builder.add_node('subgraph', subgraph)  # 직접 추가!
builder.add_edge(START, 'subgraph')
graph = builder.compile()
```

### 장단점

| 장점 | 단점 |
|------|------|
| 간단한 문법 | 상태 키 공유 필수 |
| 명확한 구조 | 유연성 낮음 |
| 자동 상태 전달 | 상태 변환 불가 |

## 다음 단계

**함수로 래핑**하여 상태 키가 달라도 서브그래프를 사용하는 방법을 배웁니다. (03번 노트북)

---
## 코드 7-3 Subgraph (함수 래핑): 상태 변환하기

# Subgraph (함수 래핑): 상태 변환하기

이 노트북에서는 **함수로 서브그래프를 래핑**하여 **상태 키가 달라도 사용**할 수 있는 방법을 배웁니다.

## 함수 래핑이 필요한 이유

```
┌────────────────────────────────────────────────────────────────────┐
│                    상태 키가 다른 경우                              │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   부모 State:                     서브그래프 State:                │
│   ┌─────────────┐                ┌─────────────┐                  │
│   │ foo: str    │                │ bar: str    │  ← 키가 다름!    │
│   └─────────────┘                │ baz: str    │                  │
│                                  └─────────────┘                  │
│                                                                    │
│   직접 연결 불가! → 함수로 래핑하여 상태 변환 필요                 │
│                                                                    │
│   foo → bar (입력 변환)                                            │
│   bar → foo (출력 변환)                                            │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 아키텍처

```
┌────────────────────────────────────────────────────────────────────┐
│                    함수 래핑 방식                                   │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   부모 그래프                                                       │
│   ┌──────────────────────────────────────────┐                     │
│   │                                          │                     │
│   │   START → ┌─────────────────────┐ → END  │                     │
│   │           │      node 함수      │        │                     │
│   │           │                     │        │                     │
│   │           │  foo → bar (변환)   │        │                     │
│   │           │  ┌───────────────┐  │        │                     │
│   │           │  │  서브그래프   │  │        │                     │
│   │           │  │   invoke()    │  │        │                     │
│   │           │  └───────────────┘  │        │                     │
│   │           │  bar → foo (변환)   │        │                     │
│   │           │                     │        │                     │
│   │           └─────────────────────┘        │                     │
│   │                                          │                     │
│   └──────────────────────────────────────────┘                     │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

# 2. State 정의 (키가 다름)

In [ ]:
from typing import TypedDict

# 부모 그래프의 상태
class State(TypedDict):
    foo: str  # 부모 그래프의 키

# 서브그래프의 상태 (부모와 키가 다름!)
class SubgraphState(TypedDict):
    bar: str  # 서브그래프의 키 (foo와 다름)
    baz: str

print("✅ State 정의 완료")
print("   - State (부모): foo")
print("   - SubgraphState: bar, baz (부모와 공유 키 없음!)")

# 3. 서브그래프 정의

In [ ]:
from langgraph.graph import StateGraph, START

def subgraph_node(state: SubgraphState):
    """
    서브그래프의 노드
    
    bar 값에 'baz' 추가
    """
    print(f"  [서브그래프] 입력 bar: {state['bar']}")
    
    new_bar = state['bar'] + 'baz'
    
    print(f"  [서브그래프] 출력 bar: {new_bar}")
    return {'bar': new_bar}


# 서브그래프 빌더
subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node('subgraph_node', subgraph_node)
subgraph_builder.add_edge(START, 'subgraph_node')

# 서브그래프 컴파일
subgraph = subgraph_builder.compile()

print("✅ 서브그래프 정의 완료")

# 4. 래퍼 함수 정의 (핵심!)

부모 상태와 서브그래프 상태 간의 변환을 담당합니다.

In [ ]:
def node(state: State):
    """
    서브그래프를 호출하는 래퍼 노드
    
    1. 부모 상태 → 서브그래프 상태 변환
    2. 서브그래프 호출
    3. 서브그래프 결과 → 부모 상태 변환
    """
    print(f"[래퍼] 부모 상태 foo: {state['foo']}")
    
    # 1. 부모 상태 → 서브그래프 상태 변환
    subgraph_input = {'bar': state['foo']}  # foo → bar
    print(f"[래퍼] 변환: foo → bar")
    
    # 2. 서브그래프 호출
    response = subgraph.invoke(subgraph_input)
    
    # 3. 서브그래프 결과 → 부모 상태 변환
    print(f"[래퍼] 변환: bar → foo")
    return {'foo': response['bar']}  # bar → foo


print("✅ 래퍼 함수 정의 완료")

# 5. 부모 그래프 정의

In [ ]:
# 부모 그래프 빌더
builder = StateGraph(State)

# 래퍼 함수를 노드로 추가 (서브그래프가 아닌 함수!)
builder.add_node('node', node)  # 래퍼 함수 추가
builder.add_edge(START, 'node')

# 부모 그래프 컴파일
graph = builder.compile()

print("✅ 부모 그래프 정의 완료")
print("   래퍼 함수가 'node' 노드로 추가됨")

In [ ]:
# 그래프 구조 시각화
print("=== 부모 그래프 구조 ===")
print(graph.get_graph().draw_mermaid())

# 6. 그래프 실행

In [ ]:
# 부모 그래프 실행
initial_state = {'foo': 'hello'}

print("=== 부모 그래프 실행 ===")
print(f"입력: {initial_state}")
print()

result = graph.invoke(initial_state)

print()
print(f"출력: {result}")
print()
print("흐름: foo('hello') → bar('hello') → bar('hellobaz') → foo('hellobaz')")

# 7. 복잡한 변환 예시

In [ ]:
# 더 복잡한 변환이 필요한 경우
class ParentState(TypedDict):
    user_input: str
    result: str

class ChildState(TypedDict):
    query: str
    answer: str

# 자식 서브그래프 노드
def child_node(state: ChildState):
    return {'answer': f"처리됨: {state['query']}"}

child_builder = StateGraph(ChildState)
child_builder.add_node('process', child_node)
child_builder.add_edge(START, 'process')
child_graph = child_builder.compile()

# 복잡한 변환 래퍼
def complex_wrapper(state: ParentState):
    # 입력 변환: user_input → query
    child_input = {
        'query': state['user_input'].upper(),  # 대문자로 변환
        'answer': ''
    }
    
    # 서브그래프 호출
    response = child_graph.invoke(child_input)
    
    # 출력 변환: answer → result (추가 가공)
    return {'result': f"[완료] {response['answer']}"}

# 부모 그래프
parent_builder = StateGraph(ParentState)
parent_builder.add_node('wrapper', complex_wrapper)
parent_builder.add_edge(START, 'wrapper')
parent_graph = parent_builder.compile()

print("✅ 복잡한 변환 예시 준비 완료")

In [ ]:
# 복잡한 변환 테스트
print("=== 복잡한 변환 테스트 ===")
result = parent_graph.invoke({'user_input': 'hello world', 'result': ''})

print(f"입력: user_input='hello world'")
print(f"출력: result='{result['result']}'")
print()
print("변환 흐름:")
print("  1. user_input='hello world'")
print("  2. query='HELLO WORLD' (대문자 변환)")
print("  3. answer='처리됨: HELLO WORLD'")
print("  4. result='[완료] 처리됨: HELLO WORLD'")

---

## 정리: Subgraph 함수 래핑

### 핵심 개념

```
┌─────────────────────────────────────────────────────────────────────┐
│                    함수 래핑 패턴                                   │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   def wrapper_node(parent_state: ParentState):                      │
│       # 1. 입력 변환                                                │
│       child_input = convert_to_child(parent_state)                  │
│                                                                     │
│       # 2. 서브그래프 호출                                          │
│       response = subgraph.invoke(child_input)                       │
│                                                                     │
│       # 3. 출력 변환                                                │
│       return convert_to_parent(response)                            │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 핵심 코드

```python
# 래퍼 함수
def node(state: State):
    # foo → bar 변환
    response = subgraph.invoke({'bar': state['foo']})
    # bar → foo 변환
    return {'foo': response['bar']}

# 부모 그래프에서 래퍼 함수를 노드로 추가
builder.add_node('node', node)  # 서브그래프가 아닌 함수!
```

### 직접 연결 vs 함수 래핑 비교

| 방식 | 상태 공유 | 변환 가능 | 복잡도 | 유연성 |
|------|----------|----------|--------|--------|
| **직접 연결** | 필수 | 불가능 | 낮음 | 낮음 |
| **함수 래핑** | 불필요 | 가능 | 높음 | 높음 |

### 사용 시나리오

| 시나리오 | 권장 방식 |
|----------|----------|
| 같은 상태 구조 | 직접 연결 |
| 다른 상태 구조 | 함수 래핑 |
| 변환/가공 필요 | 함수 래핑 |
| 재사용 서브그래프 | 함수 래핑 |

## 다음 단계

**Supervisor 패턴**을 사용하여 여러 에이전트를 관리하는 방법을 배웁니다. (04-05번 노트북)

---
## 코드 7-4~7-5 Supervisor 패턴: 에이전트 관리자

# Supervisor 패턴: 에이전트 관리자

이 노트북에서는 **Supervisor 패턴**을 사용하여 **여러 에이전트를 관리**하는 시스템을 만듭니다.

## Supervisor란?

```
┌────────────────────────────────────────────────────────────────────┐
│                    Supervisor 패턴                                 │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   Supervisor = 여러 전문 에이전트를 관리하는 관리자                │
│                                                                    │
│                    ┌──────────────┐                                │
│                    │  Supervisor  │  ← 누가 일할지 결정            │
│                    └──────┬───────┘                                │
│                           │                                        │
│              ┌────────────┼────────────┐                           │
│              │            │            │                           │
│              ▼            ▼            ▼                           │
│         ┌────────┐  ┌────────┐  ┌────────┐                         │
│         │Researcher│  │ Coder │  │ Writer │  ← 전문 에이전트       │
│         └────────┘  └────────┘  └────────┘                         │
│                                                                    │
│   역할:                                                            │
│   • Supervisor: 작업 분배, 진행 상황 모니터링, 완료 판단           │
│   • 에이전트: 각자 전문 분야의 작업 수행                           │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 아키텍처

```
┌────────────────────────────────────────────────────────────────────┐
│                    Supervisor 그래프 구조                           │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   START                                                            │
│     │                                                              │
│     ▼                                                              │
│   ┌──────────────┐                                                 │
│   │  supervisor  │ ◄─────────────────────┐                         │
│   └──────┬───────┘                       │                         │
│          │                               │                         │
│    next = ?                              │                         │
│    ╱    │    ╲                           │                         │
│   ╱     │     ╲                          │                         │
│  ▼      ▼      ▼                         │                         │
│ researcher  coder  FINISH                │                         │
│  │       │      │                        │                         │
│  │       │      └────────► END           │                         │
│  │       │                               │                         │
│  └───────┴───────────────────────────────┘                         │
│          (결과 보고)                                               │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. Supervisor 결정 스키마 정의

Supervisor가 다음에 어떤 에이전트를 호출할지 결정하는 구조를 정의합니다.

In [ ]:
from typing import Literal
from pydantic import BaseModel

class SupervisorDecision(BaseModel):
    """
    Supervisor의 결정 스키마
    
    next: 다음에 호출할 에이전트 또는 종료 신호
    """
    next: Literal['researcher', 'coder', 'FINISH']

print("✅ SupervisorDecision 스키마 정의 완료")
print("   가능한 값: researcher, coder, FINISH")

# 3. LLM 설정 (Structured Output)

In [ ]:
from langchain_ollama import ChatOllama

# 기본 모델
base_model = ChatOllama(model='llama3.2', temperature=0)

# Structured Output으로 변환 (Supervisor용)
model = base_model.with_structured_output(SupervisorDecision)

print("✅ LLM 설정 완료")
print("   with_structured_output: 항상 SupervisorDecision 형태로 응답")

# 4. 에이전트 정의

In [ ]:
# 사용 가능한 에이전트 목록
agents = ['researcher', 'coder']

print("✅ 에이전트 목록 정의")
print(f"   에이전트: {agents}")

# 5. State 정의

In [ ]:
from langgraph.graph import MessagesState

class AgentState(MessagesState):
    """
    에이전트 상태
    
    messages: 대화 기록 (MessagesState에서 상속)
    next: 다음에 호출할 에이전트
    """
    next: Literal['researcher', 'coder', 'FINISH']

print("✅ AgentState 정의 완료")

# 6. Supervisor 프롬프트 및 노드 정의

In [ ]:
# Supervisor 시스템 프롬프트
system_prompt_part_1 = f'''당신은 다음 서브에이전트 사이의 대화를 관리하는 슈퍼바이저입니다. 
서브에이전트: {agents}. 
아래 사용자 요청에 따라, 다음으로 행동할 서브에이전트를 지목하세요. 
각 서브에이전트는 임무를 수행하고 결과와 상태를 응답합니다. 
실행할 서브에이전트가 없거나 작업이 완료되면, FINISH로 응답하세요.'''

system_prompt_part_2 = f'''위 대화를 바탕으로, 다음으로 행동할 서브에이전트는 누구입니까? 
아니면 FINISH 해야 합니까? 
서브에이전트: {", ".join(agents)}, FINISH'''


def supervisor(state: AgentState):
    """
    Supervisor 노드
    
    1. 현재 대화 상태 분석
    2. 다음 에이전트 결정 (또는 FINISH)
    3. 결정 반환
    """
    messages = [
        ('system', system_prompt_part_1),
        *state['messages'],
        ('system', system_prompt_part_2),
    ]
    
    # LLM이 SupervisorDecision 형태로 응답
    decision = model.invoke(messages)
    
    return decision  # {'next': 'researcher'} 또는 {'next': 'FINISH'} 등


print("✅ Supervisor 노드 정의 완료")

# 7. 에이전트 노드 정의

In [ ]:
def researcher(state: AgentState):
    """
    Researcher 에이전트
    
    데이터 검색 및 조사 담당
    """
    response = {
        'role': 'assistant',
        'content': '관련 데이터를 찾는 중입니다... 잠시만 기다려주세요.',
    }
    
    # 실제로는 검색 도구 등을 사용
    # 여기서는 시뮬레이션
    fake_data = {
        'data': '전세계 인구 데이터: [미국: 331M, 중국: 1.4B, 인도: 1.3B]'
    }
    response['content'] += f"\n찾은 데이터: {fake_data['data']}"
    
    return {'messages': [response]}


def coder(state: AgentState):
    """
    Coder 에이전트
    
    코드 작성 담당
    """
    response = {
        'role': 'assistant',
        'content': '코드를 작성 중입니다... 잠시만 기다려주세요.',
    }
    
    # 실제로는 코드 생성 LLM 등을 사용
    # 여기서는 시뮬레이션
    fake_code = '''
def visualize_population(data):
    import matplotlib.pyplot as plt

    countries = list(data.keys())
    population = list(data.values())

    plt.bar(countries, population)
    plt.xlabel('Country')
    plt.ylabel('Population')
    plt.title('World Population by Country')
    plt.show()

data = {'USA': 331, 'China': 1400, 'India': 1300}
visualize_population(data)
'''
    response['content'] += f"\n작성된 코드:\n{fake_code}"
    
    return {'messages': [response]}


print("✅ 에이전트 노드 정의 완료")
print("   - researcher: 데이터 검색")
print("   - coder: 코드 작성")

# 8. 그래프 구성

In [ ]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(AgentState)

# 노드 추가
builder.add_node('supervisor', supervisor)
builder.add_node('researcher', researcher)
builder.add_node('coder', coder)

# 엣지 추가
builder.add_edge(START, 'supervisor')  # 시작 → Supervisor

# Supervisor의 결정에 따라 라우팅
builder.add_conditional_edges(
    'supervisor',
    lambda state: state['next'],  # next 값으로 라우팅
    {
        'researcher': 'researcher',
        'coder': 'coder',
        'FINISH': END,
    }
)

# 에이전트 → Supervisor (결과 보고)
builder.add_edge('researcher', 'supervisor')
builder.add_edge('coder', 'supervisor')

graph = builder.compile()

print("✅ 그래프 컴파일 완료")

In [ ]:
# 그래프 구조 시각화
print("=== Supervisor 그래프 구조 ===")
print(graph.get_graph().draw_mermaid())

# 9. Supervisor 에이전트 실행

In [ ]:
# 테스트 실행
initial_state = {
    'messages': [
        {
            'role': 'user',
            'content': '전세계 인구를 국적을 기준으로 시각화 해주세요.',
        }
    ],
    'next': 'supervisor',
}

print("=== Supervisor 에이전트 실행 ===")
print(f"사용자 요청: {initial_state['messages'][0]['content']}\n")

for output in graph.stream(initial_state):
    node_name, node_result = list(output.items())[0]
    
    print(f"\n{'─'*60}")
    print(f"현재 노드: {node_name}")
    
    if node_result.get('messages'):
        content = node_result['messages'][-1]
        if isinstance(content, dict):
            print(f"응답: {content['content'][:200]}..." if len(content['content']) > 200 else f"응답: {content['content']}")
        else:
            print(f"응답: {str(content)[:200]}...")
    
    if node_result.get('next'):
        print(f"다음 단계: {node_result['next']}")

---

## 정리: Supervisor 패턴

### 핵심 개념

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Supervisor 패턴 동작 원리                        │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   1. 사용자 요청 접수                                               │
│   2. Supervisor가 적절한 에이전트 선택                              │
│   3. 선택된 에이전트가 작업 수행                                    │
│   4. 결과를 Supervisor에게 보고                                     │
│   5. Supervisor가 다음 에이전트 선택 (또는 FINISH)                  │
│   6. 반복...                                                        │
│                                                                     │
│   핵심: Supervisor가 작업 흐름을 제어!                              │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 핵심 코드

```python
# 1. 결정 스키마
class SupervisorDecision(BaseModel):
    next: Literal['agent1', 'agent2', 'FINISH']

# 2. Structured Output LLM
model = base_model.with_structured_output(SupervisorDecision)

# 3. 조건부 라우팅
builder.add_conditional_edges(
    'supervisor',
    lambda state: state['next'],
    {'agent1': 'agent1', 'agent2': 'agent2', 'FINISH': END}
)

# 4. 에이전트 → Supervisor 연결
builder.add_edge('agent1', 'supervisor')
builder.add_edge('agent2', 'supervisor')
```

### Supervisor vs 다른 패턴

| 패턴 | 제어 방식 | 적합한 상황 |
|------|----------|------------|
| **Supervisor** | 중앙 집중 | 작업 순서가 중요, 복잡한 조정 |
| **Subgraph** | 계층적 | 모듈화, 재사용 |
| **Swarm** | 분산 | 병렬 처리, 독립적 작업 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)
model = model.with_structured_output(SupervisorDecision)

# 변경
base_model = ChatOllama(model='llama3.2', temperature=0)
model = base_model.with_structured_output(SupervisorDecision)
```

## ch07 요약: 고급 에이전트 패턴

| 패턴 | 용도 | 핵심 기술 |
|------|------|----------|
| **Reflection** | 자기 개선 | 역할 변환 트릭 |
| **Subgraph (직접)** | 그래프 모듈화 | 상태 키 공유 |
| **Subgraph (함수)** | 상태 변환 | 래퍼 함수 |
| **Supervisor** | 에이전트 관리 | Structured Output |

## 다음 단계

**ch08에서는** Structured Output, Streaming, Interrupt 등 **고급 기능**을 배웁니다.